## 加载配置

In [2]:
import yaml


def load_config(path="./config.yaml"):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)


cfg = load_config()
cfg

{'model_name': 'Qwen/Qwen2.5-7B-Instruct',
 'data_path': './dataclean/bedtimenews.jsonl',
 'output_dir': './qwen2.5_lora_output',
 'training': {'seed': 42,
  'num_train_epochs': 3,
  'per_device_train_batch_size': 4,
  'per_device_eval_batch_size': 4,
  'gradient_accumulation_steps': 4,
  'learning_rate': 0.0002,
  'logging_steps': 10,
  'eval_steps': 50,
  'save_steps': 100,
  'warmup_ratio': 0.03,
  'max_seq_length': 2048},
 'lora': {'r': 16,
  'lora_alpha': 32,
  'lora_dropout': 0.05,
  'target_modules': ['q_proj',
   'k_proj',
   'v_proj',
   'o_proj',
   'gate_proj',
   'up_proj',
   'down_proj']}}

## 读取数据

In [4]:
from datasets import load_dataset

print("Loading dataset...")
dataset = load_dataset("json", data_files=cfg['data_path'], split="train")
dataset = dataset.train_test_split(test_size=0.1, seed=cfg['training']['seed'])
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")

Loading dataset...
Train samples: 311
Eval samples: 35


In [6]:
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

{'messages': [{'role': 'user',
   'content': '7月31日，《美国心脏协会杂志》发表了一篇文章，认为丹参类药物会明显提高急性心衰患者的死亡率。美国人说我们的中药有问题，督工你怎么看？'},
  {'role': 'assistant',
   'content': '《美国心脏协会杂志》是行业内比较权威的杂志，这个杂志每次发关于中国的文章，国内都有大量转载。我建议各位严肃对待美国人的提醒。\n\n虽然文章发在美国，但这篇论文的第一作者是中国人，工作单位是北京协和医院、阜外医院，它的数据也来自中国，收集了189家医院，10004个急性心衰患者的病例。作者发现，在对付急性心衰的时候，99.4%的医院都要用中药。其中，使用丹参、人参、红参的患者，临床死亡率都偏高。\n\n丹参的数据是最可怕的，只要用了丹参，临床出血率就增加39%，死亡率增加36%。而且在一万个案例中，所有用了丹参类的病例都会更严重，几乎没有例外。从数据来看，丹参和红参对人体有害，这个结论的回归分析P值在0.03左右。一般P值小于0.05的观点就不可忽视，这次的结果值得中国整个医学界注意。'},
  {'role': 'user', 'content': '丹参类药物在中国使用广泛吗？'},
  {'role': 'assistant',
   'content': '中国每年生产5万吨丹参，大多数做成了心脏类药物，主要形式是注射液。这篇论文统计的是2015年数据，那年全国卖了877亿中药注射液，平均每人60块，其中参药类注射液至少176亿。现在全国中药注射液的国药批号有969个，其中和丹参有关的占207个，这绝对是最重要的中成药之一。\n\n中国有个步长药业，每年营业额100多亿，敢花50多亿给医生，去推广自己的产品，是医药代表最喜欢的单位。他家40%的利润来自丹红注射液，现在这个老板还在宣传，说丹红注射液不应该是辅助用药，按中医理论应该是主治药物。如果让他一直推广下去，每个心脏病患者都会注射丹参。\n\n步长药业的老板很推崇美国大学，今年五月，美国抓出一起大学作弊案，步长制药的董事长花了650万美元行贿，送女儿去耶鲁大学。现在这篇分析丹参类药物的文章背后也有耶鲁大学的支持，我希望赵老板能一以贯之地相信耶鲁大学。'}],
 'message': None}

## 加载tokenizer

In [ ]:
from transformers import AutoTokenizer

